**SVD**

In [3]:
import numpy as np

def getS(s,h,w, inv=False):
    '''Function to get Sigma matrix from singular values if A matrix is not square'''
    Sigma = np.zeros((h, w))
    ms = min(h,w)
    s = s[:ms]
    Sigma[:ms, :ms] = np.diag(s) if not inv else np.linalg.inv(np.diag(s))
    return Sigma

m = np.mat([[1,0,0,0,2],
            [0,0,3,0,0],
            [0,0,0,0,0],
            [0,2,0,0,0]])
h,w = m.shape
u,s,v = np.linalg.svd(m)
print('Numpy SVD Matrix reconstruction:\n',u.dot( getS(s,h,w).dot(v) ))


def SVD(A):
    h,w = A.shape
    AT_A = np.dot(A.T,A)
    ev,v = np.linalg.eig(AT_A)
    # eigenvectors goes from biggest to smallest eigenvalue
    idx = np.argsort(ev)[::-1]
    # singular value = sqrt(eigenvalue)
    s = np.sqrt(ev[ev>=0])
    s = np.sort(s)[::-1]
    v = v[:,idx]
    # if det(diag(s)) != 0 we can get U from equation A*V*inv(S) 
    # u = A.dot(v.dot(sinv.T))
    print("This is s:", s)
    U = np.zeros((h,h))
    for i,si in enumerate(s):
        gamma = (1/si) if np.sum(si) != 0 else 0
        vi = v[:,i]
        dd = A.dot(vi) if gamma != 0 else np.zeros((h,1))
        r = gamma * dd
        U[:,i:i+1] = r
    return U, s, v.conj().T


u,s,v = SVD(m)
print('Our SVD Matrix reconstruction:\n',u.dot( getS(s,h,w).dot(v) ))

Numpy SVD Matrix reconstruction:
 [[1. 0. 0. 0. 2.]
 [0. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0.]]
This is s: [3.         2.23606798 2.         0.         0.        ]
Our SVD Matrix reconstruction:
 [[1. 0. 0. 0. 2.]
 [0. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0.]]


**PCA**
To check that PCA works we will use cosine(angle) distance between all vectors in original and reduced matrices

In [4]:
from scipy.spatial.distance import pdist

m = np.mat([[1,1.4,6.7,5.1,2],
            [0.3,5,3.1,1.1,0.2],
            [0,1.2,15,0.1,0.2],
            [1,2,0,3,0]])

h,w = m.shape
n = 3
u, s, v = SVD(m)
A = u.dot( getS(s,h,w).dot(v) )
print('Our SVD Matrix reconstruction:\n',A)
print('Distance:', pdist(A, metric='cosine'))

u, s, v = SVD(m)
Ap = np.zeros((h,n))
sp = s[:n]
up = u[:,:n]
Ar = up.dot(np.diag(sp) )
print('PCA Matrix reconstruction:\n',Ar)
print('Distance:', pdist(Ar, metric='cosine'))

rmse = np.sqrt(np.mean(np.square(pdist(Ar, metric='cosine') - pdist(A, metric='cosine'))))
print ("RMSE:", rmse)

This is s: [1.71768249e+01 6.47877400e+00 4.05879049e+00 8.99106511e-01
 3.45501502e-08]
Our SVD Matrix reconstruction:
 [[ 1.00000000e+00  1.40000000e+00  6.70000000e+00  5.10000000e+00
   2.00000000e+00]
 [ 3.00000000e-01  5.00000000e+00  3.10000000e+00  1.10000000e+00
   2.00000000e-01]
 [ 1.87458936e-16  1.20000000e+00  1.50000000e+01  1.00000000e-01
   2.00000000e-01]
 [ 1.00000000e+00  2.00000000e+00 -2.29611738e-16  3.00000000e+00
  -2.28636252e-16]]
Distance: [0.35584121 0.22369064 0.4214883  0.4165221  0.39378672 0.95205148]
This is s: [1.71768249e+01 6.47877400e+00 4.05879049e+00 8.99106511e-01
 3.45501502e-08]
PCA Matrix reconstruction:
 [[ -7.72396324   3.52010578   2.38364684]
 [ -4.06255793   2.95127995  -3.2654885 ]
 [-14.76935517  -2.86095308  -0.35227386]
 [ -0.86341048   3.56205866   0.06705189]]
Distance: [0.35648135 0.22239032 0.40075648 0.41522899 0.37145461 0.95400002]
RMSE: 0.01249066950094009
